# Overview

State Space = [p,h,b]. stock price p, stock shares h, remaining balance b

Action Space = [sell, buy, hold]

Reward : Change of pf value (depending on a,s,s')

Policy

Value function

Action function

- Reward Hypothesis? Maximize pf value. 

# Env

State Space : 9 stocks * 4

1. Balance : 현금계좌 잔고

2. Stock price : p

3. Shares : 보유량

4. MACD : SMA - LMA, signal = rolling by k days (closed price)

(5) CCI : 주가가 MA에서 얼마나 떨어져 있는지 (과매수, 과매도 측정)

(6) ADX : 추세의 강도를 측정

(7) RSI

Action Space : (2k+1)^9, k <= h_max (h_max는 사용자 설정 hp)

{-k,...,-1,0,1,...,k}

hmax : 한번에 살수있는 주식의 최대 share 

## Requirements

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import datetime
import math
from numpy.random import choice
import random

import gym
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gym import spaces
from gym.utils import seeding

matplotlib.use("Agg")

import yfinance as yf

In [4]:
# Disable Warnings
import warnings
warnings.filterwarnings('ignore')

## Preprocess

In [5]:
ticker = ['XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY']
stocks = yf.download(ticker, start = "2013-01-01", end = "2023-05-09")

[*********************100%***********************]  9 of 9 completed


In [6]:
close = stocks.iloc[:,10:19]
close.columns=ticker
close.head()

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
Date,,,,,,,,,
2013-01-02,73.000000,13.704305,38.810001,29.809999,35.799999,35.560001,40.669998,48.430000,38.599998
2013-01-03,72.980003,13.688058,38.830002,29.620001,35.709999,35.560001,40.720001,48.549999,38.590000
2013-01-04,73.790001,13.850528,39.119999,29.500000,35.810001,35.740002,40.900002,48.730000,38.720001
2013-01-07,73.220001,13.818034,38.950001,29.459999,35.570000,35.389999,41.029999,48.610001,38.630001
2013-01-08,72.980003,13.777417,38.650002,29.350000,35.470001,35.279999,41.040001,48.470001,38.740002


In [7]:
# ticker
tic = []
for n in range(len(close)):
  for i in close.columns:
    tic.append(i)

# Date
Date = np.repeat(close.index,9)

# price
price = []
for r in range(len(close)):
  for c in range(close.shape[1]):
    price.append(close.iloc[r,c])

# MACD
def rolling(df,SMA, LMA, SIGNAL):
  for i in range(df.shape[1]):
    locals()[f"each_ticker_{i}"] = df.iloc[:,i].to_frame()
    locals()[f"each_ticker_{i}"][f"sma_{i}"] = df.iloc[:,i].rolling(SMA).mean()
    locals()[f"each_ticker_{i}"][f"lma_{i}"] = df.iloc[:,i].rolling(LMA).mean()
    locals()[f"each_ticker_{i}"]["macd"] = locals()[f"each_ticker_{i}"][f'sma_{i}'] - locals()[f"each_ticker_{i}"][f'lma_{i}']
    locals()[f"each_ticker_{i}"]["signal"] = locals()[f"each_ticker_{i}"].macd.rolling(SIGNAL).mean()
    locals()[f"each_ticker_{i}"].dropna(inplace=True)

  df_list = []
  for j in range(df.shape[1]):
    tmp = locals()[f"each_ticker_{j}"] # 이부분에서 코드 에러 났었음
    df_list.append(tmp)

  roll = pd.concat(df_list,axis=1)
  

  return roll

In [8]:
# Instantiate with random parameter setting
df = rolling(close,30,60,15)

# compatible for hyperparameter tuning
# also considered memory usuage

In [9]:
# After Instantiation, create instantiated dataframe
globals()['macd'] = []
for i in range(len(df)):
    tmp = list(df['macd'].iloc[i,:])
    globals()['macd'].extend(tmp)

globals()['signal'] = []
for i in range(len(df)):
    tmp = list(df['signal'].iloc[i,:])
    globals()['signal'].extend(tmp)


In [10]:
# preprocess
price = price[9*73:]
tic = tic[:22788]
Date = Date[9*73:]
# macd
# signal

data = pd.DataFrame(zip(Date, price, tic, macd, signal),columns=['Date','price','tic','macd','signal'])

### Code Error Fix

❓이 부분에서 코드 에러 났었음

⭐ Before 
```
df_list = []
for j in range(df.shape[1]):
  df_list.append(locals()[f"each_ticker_{j}])
```
이럴때 결과값은 `['each_ticker_1','each_ticker_2',...]`를 반환했음. 근데 `pd.concat()`에 넣으려면 list element가 str이 아니라 변수 그 자체가 되야함. 따라서 `pd.concat`을 사용하지 못함. 

⭐ After
```
df_list = []
for j in range(df.shape[1]):
  tmp = locals()[f"each_ticker_{j}"] 
  df_list.append(tmp)
```

**리스트 안에 데이터프레임이 담겨있는 형태. 그러니 pd.concat()의 안에 필요한 것은, <리스트로 묶인 데이터프레임> 인 것이다. 따라서 그에 맞게 만들어준다.**

In [11]:
# df_list = []
# for j in range(close.shape[1]):
#   df_list.append(f'each_ticker_{j}')

In [12]:
# for i in range(close.shape[1]):
#     globals()[f"each_ticker_{i}"] = close.iloc[:,i].to_frame()
#     globals()[f"each_ticker_{i}"][f"sma_{i}"] = close.iloc[:,i].rolling(30).mean()
#     globals()[f"each_ticker_{i}"][f"lma_{i}"] = close.iloc[:,i].rolling(60).mean()
#     globals()[f"each_ticker_{i}"]["macd"] = globals()[f"each_ticker_{i}"][f'sma_{i}'] - globals()[f"each_ticker_{i}"][f'lma_{i}']
#     globals()[f"each_ticker_{i}"]["signal"] = globals()[f"each_ticker_{i}"].macd.rolling(15).mean()
#     globals()[f"each_ticker_{i}"].dropna(inplace=True)

# Env

In [13]:
STOCK_DIM=close.shape[1]-1
INITIAL_ACCOUNT_BALANCE = 100000 # 10만 달러

In [49]:
import gym

class StockEnvTrain(gym.Env):

    metadata = {"render.modes":["human"]}

    def __init__(self, data, day = 0, render_mode=None):
      self.day = day 
      self.df = data

      # Action Space
      # normalize action space with stock_dim
      self.action_space = spaces.Box(low=-1, high=1, shape=STOCK_DIM)

      # State Space
      # Shape : 36 : [current balance] + [prices 1-9] + [shares 1-9] + [MACD 1-9]
      self.observation_space = spaces.Box(low=0, high=np.inf, shape=(36,))

      # load data
      self.data = self.df.loc[self.day,:]
      self.terminal = False

      # initialize state
      self.state = [INITIAL_ACCOUNT_BALANCE]
      self.data.macd.values.tolist()
      
      # initialize reward
      self.reward = 0
      self.cost = 0

      # history of total balance change
      self.asset_memory = [INITIAL_ACCOUNT_BALANCE]
      self.rewards_memory=[]
      self.trades = 0
      self._seed()

      assert render_mode is None or render_mode in self.metadata["render_modes"]
      self.render_mode = render_mode

      
      def _reset(self):
         pass
      def _render(self, mode='human', close=False):
         pass
      def _take_action(self, action):
         pass

In [18]:
import time
from stable_baselines3 import SAC
from stable_baselines3 import PPO
from stable_baselines3 import A2C
from stable_baselines3 import DDPG
from stable_baselines3 import TD3

from stable_baselines3.common.vec_env import DummyVecEnv

In [30]:
def train_A2C(env_train, model_name, timesteps=10000):
    """A2C Model"""
    start = time.time()
    model = A2C("MlpPolicy", env_train, verbose=2)
    model.learn(total_timesteps=timesteps)
    end = time.time()

    model.save(f"{config.TRAINED_MODEL_DIR}/{A2C_fin}")
    print("Training time (A2C): ', (end-start)/60,' minutes")
    return model

In [37]:
def train_DDPG(env_train, model_name, timesteps=10000):
 """DDPG model"""
 start = time.time()
 model = DDPG('MlpPolicy', env_train)
 model.learn(total_timesteps=timesteps)
 end = time.time()
 model.save(f'{config.TRAINED_MODEL_DIR}/{model_name}')
 print('Training time (DDPG): ', (end-start)/60,' minutes')
       
 return model
def train_PPO(env_train, model_name, timesteps=50000):
 """PPO model"""
 start = time.time()
 model = PPO2('MlpPolicy', env_train)
 model.learn(total_timesteps=timesteps)
 end = time.time()
 model.save(f'{config.TRAINED_MODEL_DIR}/{model_name}')
 print('Training time (PPO): ', (end-start)/60,' minutes')
 return model

def DRL_prediction(model, test_data, test_env, test_obs):
 """make a prediction"""
 start = time.time()
 for i in range(len(test_data.index.unique())):
   action, _states = model.predict(test_obs)
   test_obs, rewards, dones, info = test_env.step(action)
   # env_test.render()
 end = time.time()

In [50]:
train_A2C(StockEnvTrain, A2C)

Using cpu device


ValueError: The environment is of type <class 'type'>, not a Gymnasium environment. In this case, we expect OpenAI Gym to be installed and the environment to be an OpenAI Gym environment.

일단은 Environment 구축 중
gym.Env로 상속했는데 왜 안되는지 의문
Open AI gym 환경 customize하는 방법 리서치 하기

optuna로 hyperparameter tuning해서 hmax, SMA, LMA, SIGNAL optimize 해볼수 있을듯

transaction cost 구현 안되있음.


In [ ]:
import pyfolio
with pyfolio.plotting.plotting_context(font_scale=1.1):
 pyfolio.create_full_tear_sheet(returns = ensemble_strat,
 benchmark_rets=dow_strat, set_context=False)